In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import os

# 入力ディレクトリの設定
input_dir = 'data'

# ECG解析データの読み込み
print("ECG解析データの読み込み中...")
df = pd.read_parquet(os.path.join(input_dir, 'ecg_analysis.parquet'))

# データのサイズを表示
print(f"ECG解析データのサイズ: {df.shape}")

# 最初の5行を表示
print("ECG解析データの最初の5行:")
display(df.head(5))

# 基本的な統計情報を表示
print("\nECG解析データの基本統計:")
display(df.describe())


In [ ]:
# 患者データの読み込み
print("患者データの読み込み中...")
patient_df = pd.read_parquet(os.path.join(input_dir, 'patient_data.parquet'))

# 最初の5行を表示
print("患者データの最初の5行:")
display(patient_df.head(5))

# 基本的な統計情報を表示
print("\n患者データの基本統計:")
display(patient_df.describe())


In [ ]:
# 波形データの読み込みと可視化
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import os

# 波形データファイルのリスト取得
waveform_file = os.path.join(input_dir, 'waveform_data_complete.parquet')
print(f"波形データファイル: {waveform_file}")

# メモリ効率化のためにパーティションごとに読み込む
print("波形データの読み込み中...")
# 最初に基本情報だけを取得
waveform_info = pd.read_parquet(waveform_file, columns=['recording_id']).drop_duplicates(subset=['recording_id'])
print(f"記録IDの総数: {len(waveform_info)}")

# サンプルとして最初の記録IDを選択
sample_recording_id = waveform_info['recording_id'].iloc[0]
print(f"サンプル記録ID: {sample_recording_id}")

# 選択した記録IDの波形データのみを読み込む
print(f"記録ID {sample_recording_id} の波形データを読み込み中...")
sample_waveform = pd.read_parquet(
    waveform_file, 
    filters=[('recording_id', '=', sample_recording_id)]
)
print(f"サンプル波形データのサイズ: {sample_waveform.shape}")

# 最初の5行を表示
print("波形データの最初の5行:")
display(sample_waveform.head(5))

# 誘導ごとにデータをピボット
leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

# 誘導ごとに波形を可視化
plt.figure(figsize=(15, 10))
waveform_file = os.path.join(input_dir, 'waveform_data_complete.parquet')
for i, lead in enumerate(leads):
    plt.subplot(4, 3, i+1)
    lead_data = sample_waveform[sample_waveform['lead_name'] == f'"{lead}"']
    
    if not lead_data.empty:
        # sequence_numberでソート
        lead_data = lead_data.sort_values('sequence_number')
        plt.plot(lead_data['sequence_number'], lead_data['value'])
        plt.title(f'Lead {lead}')
        plt.xlabel('Sampling Point')
        plt.ylabel('Amplitude (mV)')
    else:
        plt.text(0.5, 0.5, f'No data for lead {lead}', 
                 horizontalalignment='center', verticalalignment='center')

plt.tight_layout()
plt.suptitle(f'ECG Waveform for Recording ID: {sample_recording_id}', fontsize=16)
plt.subplots_adjust(top=0.92)
plt.show()

# 全誘導を一つのグラフに表示（比較用）
plt.figure(figsize=(12, 8))
for lead in leads:
    lead_data = sample_waveform[sample_waveform['lead_name'] == f'"{lead}"']
    if not lead_data.empty:
        lead_data = lead_data.sort_values('sequence_number')
        plt.plot(lead_data['sequence_number'], lead_data['value'], label=lead)

plt.title(f'All Leads Comparison for Recording ID: {sample_recording_id}')
plt.xlabel('Sampling Point')
plt.ylabel('Amplitude (mV)')
plt.legend()
plt.grid(True)
plt.show()
